# Let's compare accuracy of some models

## Import the necessary libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from helpers.highlevel_sceneloader import HighLevelSceneLoader
from predictors.dataset_creator import TFDataSet
import tensorflow as tf
from predictors.dl_trainer import DLTrainer 
import matplotlib.pyplot as plt
import pickle

## Import the data

In [3]:
ROOT = os.getcwd()

rel_p_img_b = 'helpers/analysed_vars_storage/img_bounds.xml'
rel_p_dests = 'helpers/analysed_vars_storage/destination_locations.xml'
p_img_bounds = os.path.join(ROOT, rel_p_img_b)
p_dest_locs = os.path.join(ROOT, rel_p_dests)

#TODO: older version of OpenTraj needed: "git checkout d249ba6951dd0f54b532fbe2ca6edc46b0d7093f"
opentraj_root = os.path.join(ROOT, 'OpenTraj')
root_datasets = os.path.join(ROOT, 'data/path_data')
sys.path.append(opentraj_root) # add package to pythonpath

scene_data = HighLevelSceneLoader(p_img_bounds, p_dest_locs)
scene_data.load_ind(root_datasets, 11)

## Global Parameters

In [4]:
MAX_EPOCHS = 50
PATIENCE = 10
LSTM_LAYER_SIZE = 64
DENSE_LAYER_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_DENSE_LAYERS = 2
DS_LOCATION = "data/pickle/ds_creation_d/calc_before_all_f.pickle"

NOISE_STD = .15
N_REPEATS = 7
SEQ_IN_LENGTH = 5
SEQ_OUT_LENGTH = 1

## Baseline: no extra features

### Create Dataset 

In [ ]:
extra_features_dict = {}

# Load data in order to not need to do calculations again
with open(DS_LOCATION, 'rb') as handle:
    my_ds_creation_dict = pickle.load(handle)

my_ds = TFDataSet.init_as_fixed_length(scene_data.traj_dataframe, scale_list=["pos_x", "pos_y"], seq_in_length=SEQ_IN_LENGTH, label_length=SEQ_OUT_LENGTH, seq_stride=1,
extra_features_dict=extra_features_dict, ds_creation_dict=my_ds_creation_dict, noise_std=NOISE_STD, n_repeats = N_REPEATS)

In [ ]:
next(iter(my_ds.tf_ds_dict["train"]))

### Create Model

In [13]:
my_trainer_basic = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
network = my_trainer_basic.LSTM_one_shot_predictor_named_i(my_ds, LSTM_LAYER_SIZE, DENSE_LAYER_SIZE, 
NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=[])

save_path = "data/model_weights/m_no_extra_f.h5"

try:
    my_trainer_basic.load_weights(save_path)
except:
    my_trainer_basic.compile_and_fit(my_ds, save_path)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Total params: 61,826
Trainable params: 61,826
Non-trainable params: 0
__________________________________________________________________________________________________
None
Epoch 1/50


/home/bram/anaconda3/envs/thesis_basic/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:588: UserWarning: Input dict contained keys ['labels', 'all_points', 'n_destinations', 'n_points'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


924/924 [==============================] - 12s 10ms/step - loss: 0.0459 - mean_absolute_error: 0.1696 - val_loss: 0.0170 - val_mean_absolute_error: 0.1020
Epoch 2/50
924/924 [==============================] - 9s 9ms/step - loss: 0.0066 - mean_absolute_error: 0.0605 - val_loss: 0.0052 - val_mean_absolute_error: 0.0523
Epoch 3/50
924/924 [==============================] - 8s 8ms/step - loss: 0.0034 - mean_absolute_error: 0.0435 - val_loss: 0.0032 - val_mean_absolute_error: 0.0410
Epoch 4/50
924/924 [==============================] - 8s 8ms/step - loss: 0.0025 - mean_absolute_error: 0.0381 - val_loss: 0.0036 - val_mean_absolute_error: 0.0487
Epoch 5/50
924/924 [==============================] - 8s 8ms/step - loss: 0.0024 - mean_absolute_error: 0.0374 - val_loss: 0.0021 - val_mean_absolute_error: 0.0361
Epoch 6/50
924/924 [==============================] - 8s 8ms/step - loss: 0.0025 - mean_absolute_error: 0.0381 - val_loss: 0.0026 - val_mean_absolute_error: 0.0408
Epoch 7/50
924/924 [=====

## Added Feature: All Destinations

In [ ]:
my_trainer_all_dests = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
network = my_trainer_all_dests.LSTM_one_shot_predictor_named_i(my_ds, LSTM_LAYER_SIZE, DENSE_LAYER_SIZE, 
NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=["all_destinations"])

save_path = "data/model_weights/m_all_dests.h5"

my_trainer_all_dests.load_weights(save_path)
try:
    my_trainer_all_dests.load_weights(save_path)
except:
    my_trainer_all_dests.compile_and_fit(my_ds, save_path)

## Added feature: All points

In [ ]:
my_trainer_all_points = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
network = my_trainer_all_points.LSTM_one_shot_predictor_named_i(my_ds, LSTM_LAYER_SIZE, DENSE_LAYER_SIZE, 
NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=["all_points"])

save_path = "data/model_weights/m_all_points.h5"

my_trainer_all_points.load_weights(save_path)
try:
    my_trainer_all_points.load_weights(save_path)
except:
    my_trainer_all_points.compile_and_fit(my_ds, save_path)